In [60]:
import pandas as pd
import numpy as np
import scipy.spatial.distance as scidist
import json

In [63]:
from math import radians, degrees, sin, cos, asin, acos, sqrt

In [64]:
def great_circle_dist(lonlat1,lonlat2):
    '''https://medium.com/@petehouston/calculate-distance-of-two-locations-on-earth-using-python-1501b1944d97'''
    lon1,lat1=lonlat1
    lon2,lat2=lonlat2
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    return 6371 * (
        acos(sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(lon1 - lon2))
    )

In [65]:
hs = pd.read_csv('OEFFHALTESTOGD.csv')

In [66]:
# LTYP 4 = subway
hs = hs.loc[hs['LTYP'] == 4]

In [67]:
hs.drop(labels=['FID','OBJECTID','HTXT','DIVA_ID','WEBLINK1','SE_ANNO_CAD_DATA','LTYP'], axis=1,inplace=True)

In [68]:
hs.reset_index(inplace=True)

In [69]:
hs["LINES"] =hs['HLINIEN'].str.findall(r'(?P<LINE>U\d{1}[EZ]?)').str.join(',')

In [70]:
hspos = hs.SHAPE.str.extract("POINT \((?P<lon>\d{2}\.\d{10,15}) (?P<lat>\d{2}\.\d{10,15})\)")

In [71]:
hs = pd.concat([hs, hspos],axis=1)

In [72]:
hs.drop(labels=['SHAPE', 'HLINIEN'], axis=1,inplace=True)
hs.reset_index(inplace=True)

In [74]:
hs['lat'] = hs['lat'].astype('float64')

In [75]:
hs['lon'] = hs['lon'].astype('float64')

In [76]:
hs_dist = scidist.squareform(scidist.pdist(hs[['lon','lat']], great_circle_dist))

In [88]:
with open('wienerlinien.json', 'w') as f:
    jsonstr = json.dumps({
        'E': hs['HTXTK'].tolist(),
        'EA': hs_dist.tolist(),
        'SR': hs['LINES'].str.split(',').tolist()
    })
    f.write(jsonstr)
    f.flush()

['Alaudagasse',
 'Braunschweiggasse',
 'Burggasse-Stadthalle',
 'Donauinsel',
 'Stubentor',
 'Alser Straße',
 'Alte Donau',
 'Altes Landgut',
 'Am Schöpfwerk',
 'Neubaugasse',
 'Siebenhirten',
 'Handelskai',
 'Mitte-Landstraße',
 'Simmering',
 'Gasometer',
 'Donaumarina',
 'Erlaaer Straße',
 'Erdberg',
 'Donaustadtbrücke',
 'Leopoldau',
 'Donauspital',
 'Stadlau',
 'Stadtpark',
 'Aspernstraße',
 'Längenfeldgasse',
 'Margaretengürtel',
 'Kendlerstraße',
 'Stephansplatz',
 'Floridsdorf',
 'Meidling Hauptstraße',
 'Hauptbahnhof',
 'Aderklaaer Straße',
 'Großfeldsiedlung',
 'Rennbahnweg',
 'Friedensbrücke',
 'Michelbeuern-AKH',
 'Taubstummengasse',
 'Thaliastraße',
 'Tscherttegasse',
 'Unter St. Veit',
 'Taborstraße',
 'Messe-Prater',
 'Krieau',
 'Stadion',
 'Zippererstraße',
 'Aspern Nord',
 'Nestroyplatz',
 'Volkstheater',
 'Vorgartenstraße',
 'Enkplatz',
 'Gumpendorfer Straße',
 'Niederhofstraße',
 'Seestadt',
 'Oberlaa',
 'Nußdorfer Straße',
 'Ober St. Veit',
 'Hardeggasse',
 'Ottakrin